In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters.character import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

from os import listdir
from os.path import join

In [2]:
prompt = PromptTemplate(
    template="""You are an assistant for question-answering tasks.
    Use the following documents to answer the question.
    If you don't know the answer, just say that you don't know.
    Question: {question}
    Documents: {documents}
    Answer:
    """,
    input_variables=["question", "documents"],
)

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250,
    chunk_overlap=0
)

def load_documents(folder):
    files = [file for file in listdir("library") if file.endswith(".pdf")]
    docs = [PyPDFLoader(join(folder, file)).load() for file in files]
    return [item for sublist in docs for item in sublist]

In [3]:
class RAGApplication:

    def __init__(self, folder="library"):

        self.llm = ChatOllama(model="llama3.1:8b", temperature=0)

        self.rag_chain = prompt | self.llm | StrOutputParser()

        self.docs_list = load_documents(folder=folder)

        self.vectorstore = SKLearnVectorStore.from_documents(
            documents=text_splitter.split_documents(self.docs_list),
            embedding=OllamaEmbeddings(model="qwen3-embedding:8b"),
        )

        self.retriever = self.vectorstore.as_retriever(
            search_type="mmr",
            search_kwargs={
                "k": 5,
            }
        )

    def ask(self, question):

        documents = self.retriever.invoke(question)
        documents = [doc.page_content for doc in documents]
        documents = "\\n".join(documents)

        answer = self.rag_chain.invoke({
            "question": question,
            "documents": documents
        })

        return answer

In [4]:
rag_app = RAGApplication()

In [5]:
rag_app.ask("Welchen Umfang dürfen Bachelor- und Masterarbeiten haben?")

'Die Bachelor- und Masterarbeiten dürfen einen Umfang von 10.000 bis 15.000 Wörtern (+/- 10% Abweichung erlaubt) haben. Im inhaltlich begründeten Einzelfall kann mit Zustimmung der Studiengangsleitung von diesem Umfang abgewichen werden.'